## Conjugate Gradient Method

In [1]:
import numpy as np
import matplotlib.image as mpimg
from scipy.ndimage import gaussian_filter


def laplace_dirichlet(u: np.ndarray) -> np.ndarray:
    up = np.pad(u, ((1, 1), (1, 1)), mode="constant", constant_values=0)
    out = (
        -4.0 * up[1:-1, 1:-1]
        + up[0:-2, 1:-1]
        + up[2:, 1:-1]
        + up[1:-1, 0:-2]
        + up[1:-1, 2:]
    )
    return out

def M_matrix(u, v, Ix, Iy, lam):
    Mu = (Ix * Ix) * u + (Ix * Iy) * v - lam * laplace_dirichlet(u)
    Mv = (Ix * Iy) * u + (Iy * Iy) * v - lam * laplace_dirichlet(v)
    return Mu, Mv

def derivatives(I0, I1):
    I0x = np.zeros_like(I0)
    I1x = np.zeros_like(I1)
    I0x[:, :-1] = I0[:, 1:] - I0[:, :-1]
    I0x[:, -1]  = I0[:, -1] - I0[:, -2]
    I1x[:, :-1] = I1[:, 1:] - I1[:, :-1]
    I1x[:, -1]  = I1[:, -1] - I1[:, -2]
    Ix = 0.5 * (I0x + I1x)

    I0y = np.zeros_like(I0)
    I1y = np.zeros_like(I1)
    I0y[:-1, :] = I0[1:, :] - I0[:-1, :]
    I0y[-1, :]  = I0[-1, :] - I0[-2, :]
    I1y[:-1, :] = I1[1:, :] - I1[:-1, :]
    I1y[-1, :]  = I1[-1, :] - I1[-2, :]
    Iy = 0.5 * (I0y + I1y)

    return Ix, Iy


def OF_cg(u0, v0, Ix, Iy, reg, rhsu, rhsv, tol=1e-8, maxit=2000, verbose=False):
    u, v = u0.copy(), v0.copy()
    Au, Av = M_matrix(u, v, Ix, Iy, reg)
    ru, rv = rhsu - Au, rhsv - Av

    def dot2(a1, a2, b1, b2):
        return float(np.sum(a1 * b1) + np.sum(a2 * b2))

    norm_r0 = np.sqrt(dot2(ru, rv, ru, rv))
    if norm_r0 == 0.0:
        return u, v, {"it": 0, "relres": 0.0, "converged": True}

    pu, pv = ru.copy(), rv.copy()
    relres, it = 1.0, 0

    for k in range(1, maxit + 1):
        Apu, Apv = M_matrix(pu, pv, Ix, Iy, reg)
        denom = dot2(pu, pv, Apu, Apv)
        rr = dot2(ru, rv, ru, rv)
        alpha = rr / denom
        u += alpha * pu
        v += alpha * pv
        ru -= alpha * Apu
        rv -= alpha * Apv

        relres = np.sqrt(dot2(ru, rv, ru, rv)) / norm_r0
        if verbose and (k % 10 == 0 or relres < tol):
            print(f"it={k:4d} relres={relres:.3e}")
        if relres < tol:
            it = k
            break

        rr_new = dot2(ru, rv, ru, rv)
        beta = rr_new / rr
        pu, pv = ru + beta * pu, rv + beta * pv
        it = k

    return u, v, {"it": it, "relres": relres, "converged": relres < tol}

def build_right_hand_side(Ix, Iy, It):
    return -Ix * It, -Iy * It

def run_optical_flow(frame0_path, frame1_path, lam=1.0, sigma=1.0, tol=1e-8, maxit=2000):
    I0 = mpimg.imread(frame0_path).astype(float)
    I1 = mpimg.imread(frame1_path).astype(float)
    if I0.ndim == 3:
        I0 = np.mean(I0, axis=2)
        I1 = np.mean(I1, axis=2)

    I0 = gaussian_filter(I0, sigma)
    I1 = gaussian_filter(I1, sigma)

    Ix,Iy = derivatives(I0,I1)
    It = I1 - I0

    rhsu, rhsv = build_right_hand_side(Ix, Iy, It)
    u0, v0 = np.zeros_like(I0), np.zeros_like(I0)

    u, v, info = OF_cg(u0, v0, Ix, Iy, lam, rhsu, rhsv, tol, maxit, verbose=True)

    print(f"CG finished in {info['it']} iterations, relres={info['relres']:.2e}, converged={info['converged']}")
    return u, v, info


frame0_path = "/Users/ethanclement/Documents/NLAProject/frame10.png"
frame1_path = "/Users/ethanclement/Documents/NLAProject/frame11.png"

print(run_optical_flow(frame0_path ,frame1_path ))


it=  10 relres=1.436e+00
it=  20 relres=1.331e+00
it=  30 relres=1.210e+00
it=  40 relres=1.052e+00
it=  50 relres=8.778e-01
it=  60 relres=7.130e-01
it=  70 relres=5.967e-01
it=  80 relres=4.992e-01
it=  90 relres=4.154e-01
it= 100 relres=3.554e-01
it= 110 relres=3.082e-01
it= 120 relres=2.619e-01
it= 130 relres=2.096e-01
it= 140 relres=1.803e-01
it= 150 relres=1.507e-01
it= 160 relres=1.246e-01
it= 170 relres=9.923e-02
it= 180 relres=8.198e-02
it= 190 relres=7.041e-02
it= 200 relres=5.994e-02
it= 210 relres=5.189e-02
it= 220 relres=4.611e-02
it= 230 relres=3.973e-02
it= 240 relres=3.315e-02
it= 250 relres=2.804e-02
it= 260 relres=2.301e-02
it= 270 relres=1.887e-02
it= 280 relres=1.565e-02
it= 290 relres=1.289e-02
it= 300 relres=1.056e-02
it= 310 relres=8.685e-03
it= 320 relres=7.409e-03
it= 330 relres=6.280e-03
it= 340 relres=5.562e-03
it= 350 relres=4.698e-03
it= 360 relres=3.862e-03
it= 370 relres=3.240e-03
it= 380 relres=2.856e-03
it= 390 relres=2.458e-03
it= 400 relres=2.074e-03


In [ ]:
import numpy as np
import matplotlib.image as mpimg
from scipy.ndimage import gaussian_filter

def laplace_dirichlet(u: np.ndarray) -> np.ndarray:
    """
    This function computes the 5-point discrete Laplacian of a 2D array u
    Discrete Laplacian is delta(u,j) = -4u(i,j)+u(i-1,j)+u(i+1,j)+u(i,j-1)+u(i,j+1)
    """
    up = np.pad(u, ((1, 1), (1, 1)), mode="constant", constant_values=0)
    out = (
        -4.0 * up[1:-1, 1:-1] # interior
        + up[0:-2, 1:-1] # pixel above
        + up[2:, 1:-1] # pixel below
        + up[1:-1, 0:-2] # pixel left
        + up[1:-1, 2:] # pixel right 
    )

    # padding is for u = 0 on the boundary
    return out

def M_matrix(u, v, Ix, Iy, lam):
    """
    M_matrix applies the Horn–Schunck linearized optical flow operator to a pair of unknown fields u and v.

    (Ix * u + Iy * v) * Ix - lambda * Laplacian(u) = -Ix * It
    (Ix * u + Iy * v) * Iy - lambda * Laplacian(v) = -Iy * It

    = Ix*Ix*u + Ix*Iy*v - lam*laplace(u)
    = Ix*Iy*u + Iy*Iy*v - lam*laplace(v)

    See equation (2)
    """
    Mu = (Ix * Ix) * u + (Ix * Iy) * v - lam * laplace_dirichlet(u)
    Mv = (Ix * Iy) * u + (Iy * Iy) * v - lam * laplace_dirichlet(v)

    return Mu, Mv

def derivatives(I0, I1):
    """
    Forward finite difference method

    I(x_i,y_i)/Iy = 1/2(I0(x_i,y_i) + I1(x_i, y_i))

    I0x(i,j) = I0(i,j+1) - I0(i,j) for j = 1,2,...m-1 except last column

    See 2. Discretisation
    """
    I0x = np.zeros_like(I0)
    I1x = np.zeros_like(I1)
    I0x[:, :-1] = I0[:, 1:] - I0[:, :-1]
    I0x[:, -1]  = I0[:, -1] - I0[:, -2]
    I1x[:, :-1] = I1[:, 1:] - I1[:, :-1]
    I1x[:, -1]  = I1[:, -1] - I1[:, -2]
    Ix = 0.5 * (I0x + I1x)

    I0y = np.zeros_like(I0)
    I1y = np.zeros_like(I1)
    I0y[:-1, :] = I0[1:, :] - I0[:-1, :]
    I0y[-1, :]  = I0[-1, :] - I0[-2, :]
    I1y[:-1, :] = I1[1:, :] - I1[:-1, :]
    I1y[-1, :]  = I1[-1, :] - I1[-2, :]
    Iy = 0.5 * (I0y + I1y)

    return Ix, Iy

def build_right_hand_side(Ix, Iy, It):
    """
    Right hand side of the linearized Horn-Schunck equations

    See equation (5)
    """
    
    return -Ix * It, -Iy * It


def OF_cg(u0, v0, Ix, Iy, reg, rhsu, rhsv, tol=1e-8, maxit=2000, verbose=False):
    """
    The CG method for the optimal flow problem

    input:
    u0 - initial guess for u
    v0 - initial guess for v
    Ix - x-derivative of the first frame
    Iy - y-derivative of the first frame
    reg - regularisation parameter lambda
    rhsu - right-hand side in the equation for u
    rhsv - right-hand side in the equation for v
    tol - relative residual tolerance
    maxit - maximum number of iterations

    output:
    u - numerical solution for u
    v - numerical solution for v

    A is the 2x2 block matrix ( Ix^2 - lambda * laplacian,   IxIy; 
                                IxIy,                       Iy^2 - lambda * laplacian )
    """

    # Initialization of CG, we start with the initial guess u0, v0.as_integer_ratio
    # We apply the operator A(u0, v0) = (Au0, Av0)
    # We compute the initial residual r0 = b - Au0 
    u, v = u0.copy(), v0.copy()
    Au, Av = M_matrix(u, v, Ix, Iy, reg)
    ru, rv = rhsu - Au, rhsv - Av

    # This is the dot product over 2 fields
    def dot2(a1, a2, b1, b2):
        return float(np.sum(a1 * b1) + np.sum(a2 * b2))

    # Basically checks if the residual is zero, in which case the initial guess would already be the solution
    norm_r0 = np.sqrt(dot2(ru, rv, ru, rv))
    if norm_r0 == 0.0:
        return u, v, {"it": 0, "relres": 0.0, "converged": True}

    # Initial search condition such that p0 = r0
    pu, pv = ru.copy(), rv.copy()
    relres, it = 1.0, 0

    # That's the main CG loop
    # We apply Apu, Apv = M_matrix(pu, pv, Ix, Iy, reg)

    for k in range(1, maxit + 1):
        Apu, Apv = M_matrix(pu, pv, Ix, Iy, reg)
        denom = dot2(pu, pv, Apu, Apv)
        rr = dot2(ru, rv, ru, rv)
        alpha = rr / denom
        u += alpha * pu
        v += alpha * pv
        ru -= alpha * Apu
        rv -= alpha * Apv

        relres = np.sqrt(dot2(ru, rv, ru, rv)) / norm_r0
        if verbose and (k % 10 == 0 or relres < tol):
            print(f"it={k:4d} relres={relres:.3e}")
        if relres < tol:
            it = k
            break

        rr_new = dot2(ru, rv, ru, rv)
        beta = rr_new / rr
        pu, pv = ru + beta * pu, rv + beta * pv
        it = k

    return u, v, {"it": it, "relres": relres, "converged": relres < tol}

def zero_boundary(a):
    """Enforce homogeneous Dirichlet boundary conditions."""
    a = a.copy()
    a[0, :]  = 0.0
    a[-1, :] = 0.0
    a[:, 0]  = 0.0
    a[:, -1] = 0.0
    return a

def laplacian(u):
    """
    5-point Laplacian on interior points only (no 1/h^2 factor),
    boundaries left untouched (assumed Dirichlet = 0).
    """
    n, m = u.shape
    L = np.zeros_like(u)
    L[1:n-1, 1:m-1] = (
        u[0:n-2, 1:m-1] + u[2:n, 1:m-1] +
        u[1:n-1, 0:m-2] + u[1:n-1, 2:m] -
        4.0 * u[1:n-1, 1:m-1]
    )
    return L

def A(u, v, Ix, Iy, reg, level=0):
    """
    Multigrid operator A(u,v):

    A(u) = Ix^2 u + Ix Iy v - reg * (1/h^2) * Laplacian(u)
    with h = 2^level  ->  1/h^2 = 4^(-level)
    """
    u = zero_boundary(u)
    v = zero_boundary(v)

    h2inv = 4.0 ** (-level)  # 1/h^2 for grid spacing h = 2^level

    Lu = laplacian(u)
    Lv = laplacian(v)

    Ix2  = Ix * Ix
    Iy2  = Iy * Iy
    IxIy = Ix * Iy

    Au = Ix2 * u + IxIy * v - reg * (h2inv * Lu)
    Av = IxIy * u + Iy2 * v - reg * (h2inv * Lv)

    return zero_boundary(Au), zero_boundary(Av)

def smoothing(u0, v0, Ix, Iy, reg, rhsu, rhsv, s1, level=0, parity=0):
    """
    Red–black Gauss–Seidel smoother with local 2×2 solves on each pixel.
    - parity=0: RB; parity=1: BR (for symmetric preconditioner)
    """
    u = u0.copy()
    v = v0.copy()
    n, m = u.shape

    eps = 1e-10
    h2inv = 4.0 ** (-level)    # 1/h^2
    gamma = reg * h2inv        # multiplies Laplacian neighbor contributions

    sweeps = max(1, s1 - int(level))

    # local 2×2 block coeffs
    a = Ix * Ix + 4.0 * gamma
    b = Ix * Iy
    c = Iy * Iy + 4.0 * gamma
    det = a * c - b * b
    det[np.abs(det) < eps] = eps

    # checkerboard parity mask
    I_idx = np.arange(n)
    J_idx = np.arange(m)
    par = (I_idx[:, None] + J_idx[None, :]) & 1  # 0/1

    for _ in range(sweeps):
        for p in (parity, 1 - parity):
            mask = (par == p)

            Su = (u[:-2, 1:-1] + u[2:, 1:-1] +
                  u[1:-1, :-2] + u[1:-1, 2:])
            Sv = (v[:-2, 1:-1] + v[2:, 1:-1] +
                  v[1:-1, :-2] + v[1:-1, 2:])

            fu = rhsu[1:-1, 1:-1] + gamma * Su
            fv = rhsv[1:-1, 1:-1] + gamma * Sv

            a_loc = a[1:-1, 1:-1]
            b_loc = b[1:-1, 1:-1]
            c_loc = c[1:-1, 1:-1]
            det_loc = det[1:-1, 1:-1]

            u_new = (c_loc * fu - b_loc * fv) / det_loc
            v_new = (-b_loc * fu + a_loc * fv) / det_loc

            inner_mask = mask[1:-1, 1:-1]
            u[1:-1, 1:-1][inner_mask] = u_new[inner_mask]
            v[1:-1, 1:-1][inner_mask] = v_new[inner_mask]

        u = zero_boundary(u)
        v = zero_boundary(v)

    return u, v

def residual_MG(u, v, Ix, Iy, reg, rhsu, rhsv, level=0):
    """Residual r = b - A(u,v) for the multigrid operator."""
    Au, Av = A(u, v, Ix, Iy, reg, level=level)
    rhu = zero_boundary(rhsu.copy()) - Au
    rhv = zero_boundary(rhsv.copy()) - Av
    return rhu, rhv

def restrict_block_2x2(f: np.ndarray) -> np.ndarray:
    """
    2×2 block-average restriction (1/4 stencil):
    fine (n_f, m_f) -> coarse (n_f/2, m_f/2).

    Assumes n_f and m_f are even.
    """
    n_f, m_f = f.shape
    if n_f % 2 != 0 or m_f % 2 != 0:
        raise ValueError("restrict_block_2x2: fine grid dimensions must be even.")

    n_c = n_f // 2
    m_c = m_f // 2

    coarse = 0.25 * (
        f[0:n_f:2, 0:m_f:2] +  # top-left
        f[1:n_f:2, 0:m_f:2] +  # bottom-left
        f[0:n_f:2, 1:m_f:2] +  # top-right
        f[1:n_f:2, 1:m_f:2]    # bottom-right
    )

    return coarse

def restriction_MG(rhu, rhv, Ix, Iy):
    """
    Restriction for multigrid using 2×2 block averaging (1/4 stencil)
    for both residuals and coefficient fields.
    """
    r2hu = restrict_block_2x2(rhu)
    r2hv = restrict_block_2x2(rhv)
    Ix2h = restrict_block_2x2(Ix)
    Iy2h = restrict_block_2x2(Iy)
    return r2hu, r2hv, Ix2h, Iy2h

def prolong_block_2x2(ec: np.ndarray) -> np.ndarray:
    """
    2×2 injection prolongation:
    coarse (n_c, m_c) -> fine (2*n_c, 2*m_c),
    each coarse value copied into a 2×2 block.
    """
    n_c, m_c = ec.shape
    n_f = 2 * n_c
    m_f = 2 * m_c

    ef = np.zeros((n_f, m_f), dtype=ec.dtype)
    ef[0::2, 0::2] = ec
    ef[1::2, 0::2] = ec
    ef[0::2, 1::2] = ec
    ef[1::2, 1::2] = ec
    return ef

def prolongation_MG(e2hu, e2hv):
    """Prolongation for multigrid using 2×2 injection."""
    ehu = prolong_block_2x2(e2hu)
    ehv = prolong_block_2x2(e2hv)
    return ehu, ehv

def OF_cg_level(u0, v0, Ix, Iy, reg, rhsu, rhsv,
                tol=1e-8, maxit=2000, level=0, verbose=False):
    """CG solver using the multigrid operator A on a given level."""
    u = zero_boundary(u0.copy())
    v = zero_boundary(v0.copy())

    Au, Av = A(u, v, Ix, Iy, reg, level)
    ru = zero_boundary(rhsu.copy()) - Au
    rv = zero_boundary(rhsv.copy()) - Av

    def dot2(a1, a2, b1, b2):
        return float(np.vdot(a1, b1) + np.vdot(a2, b2))

    r2_0 = dot2(ru, rv, ru, rv)
    if r2_0 == 0.0:
        return u, v, {"it": 0, "relres": 0.0, "converged": True}

    r0_norm = np.sqrt(r2_0)
    pu, pv = ru.copy(), rv.copy()
    relres = 1.0
    it = 0

    for k in range(1, maxit + 1):
        Apu, Apv = A(pu, pv, Ix, Iy, reg, level)
        denom = dot2(pu, pv, Apu, Apv)
        rr = dot2(ru, rv, ru, rv)
        alpha = rr / denom

        u += alpha * pu
        v += alpha * pv

        ru -= alpha * Apu
        rv -= alpha * Apv

        relres = np.sqrt(dot2(ru, rv, ru, rv)) / r0_norm
        if verbose and (k % 10 == 0 or relres < tol):
            print(f"[L{level}] CG it={k:3d} relres={relres:.3e}")
        if relres < tol:
            it = k
            break

        rr_new = dot2(ru, rv, ru, rv)
        beta = rr_new / rr
        pu = ru + beta * pu
        pv = rv + beta * pv
        it = k

    return u, v, {"it": it, "relres": relres, "converged": relres < tol}

def V_cycle(u0, v0, Ix, Iy, reg, rhsu, rhsv,
            s1, s2, level, max_level, parity=0):
    """
    Recursive V-cycle for the optical flow problem.
    Uses:
      - RB/BR GS smoother
      - 2×2 block restriction (1/4 stencil)
      - 2×2 injection prolongation
      - CG on the coarsest residual equation
    """
    # Pre-smoothing
    u, v = smoothing(u0, v0, Ix, Iy, reg, rhsu, rhsv, s1, level=level, parity=0)

    # Residual on current level
    rhu, rhv = residual_MG(u, v, Ix, Iy, reg, rhsu, rhsv, level=level)

    # Restrict to coarse grid
    r2hu, r2hv, Ix2h, Iy2h = restriction_MG(rhu, rhv, Ix, Iy)

    if level == max_level - 1:
        # Coarsest: solve A e = r on the restricted grid
        eu0 = np.zeros_like(r2hu)
        ev0 = np.zeros_like(r2hv)
        eu, ev, _ = OF_cg_level(
            eu0, ev0, Ix2h, Iy2h, reg, r2hu, r2hv,
            tol=1e-8, maxit=200, level=level+1, verbose=False
        )
    else:
        # Recursive V-cycle on coarser grid
        eu0 = np.zeros_like(r2hu)
        ev0 = np.zeros_like(r2hv)
        eu, ev = V_cycle(
            eu0, ev0, Ix2h, Iy2h, reg, r2hu, r2hv,
            s1, s2, level+1, max_level, parity=parity
        )

    # Prolongate error and correct
    ehu, ehv = prolongation_MG(eu, ev)
    u += ehu
    v += ehv

    # Post-smoothing with reversed parity
    u, v = smoothing(u, v, Ix, Iy, reg, rhsu, rhsv, s2, level=level, parity=1)

    return u, v

def OF_pcg(u0, v0, Ix, Iy, reg, rhsu, rhsv,
           tol=1e-8, maxit=2000, level=0,
           s1=2, s2=2, max_level=3, verbose=False):
    """
    Preconditioned CG using a symmetric multigrid V-cycle as preconditioner.
    """
    u = zero_boundary(u0.copy())
    v = zero_boundary(v0.copy())

    Au, Av = A(u, v, Ix, Iy, reg, level)
    ru = zero_boundary(rhsu.copy()) - Au
    rv = zero_boundary(rhsv.copy()) - Av

    def dot2(a1, a2, b1, b2):
        return float(np.vdot(a1, b1) + np.vdot(a2, b2))

    r2_0 = dot2(ru, rv, ru, rv)
    if r2_0 == 0.0:
        return u, v, {"it": 0, "relres": 0.0, "converged": True, "res_hist": [0.0]}

    r0_norm = np.sqrt(r2_0)
    res_hist = [1.0]

    # z0 = M^{-1} r0 via V-cycle
    zu, zv = V_cycle(
        np.zeros_like(ru), np.zeros_like(rv),
        Ix, Iy, reg, ru, rv,
        s1, s2, level, max_level, parity=0
    )

    pu = zu.copy()
    pv = zv.copy()

    rz_old = dot2(ru, rv, zu, zv)
    rel = 1.0
    it = 0

    while it < maxit and rel > tol:
        Apu, Apv = A(pu, pv, Ix, Iy, reg, level)
        denom = dot2(pu, pv, Apu, Apv)
        alpha = rz_old / denom

        u += alpha * pu
        v += alpha * pv

        ru -= alpha * Apu
        rv -= alpha * Apv

        r2_new = dot2(ru, rv, ru, rv)
        rel = np.sqrt(r2_new) / r0_norm
        res_hist.append(rel)

        if verbose and (it % 10 == 0 or rel <= tol):
            print(f"[PCG] it={it:4d} relres={rel:.3e}")

        if rel <= tol:
            break

        # New preconditioned residual
        zu, zv = V_cycle(
            np.zeros_like(ru), np.zeros_like(rv),
            Ix, Iy, reg, ru, rv,
            s1, s2, level, max_level, parity=0
        )

        rz_new = dot2(ru, rv, zu, zv)
        beta = rz_new / rz_old

        pu = zu + beta * pu
        pv = zv + beta * pv

        rz_old = rz_new
        it += 1

    return u, v, {"it": it, "relres": rel, "converged": rel <= tol, "res_hist": res_hist}

def pad_to_multiple_of_2L(im: np.ndarray, max_level: int) -> np.ndarray:
    """
    Pad image so each dimension is divisible by 2**max_level.
    Padding uses edge mode.
    """
    n, m = im.shape
    k = 2 ** max_level

    def next_multiple(x, k):
        return ((x + k - 1) // k) * k

    n_target = next_multiple(n, k)
    m_target = next_multiple(m, k)

    pad_n = n_target - n
    pad_m = m_target - m

    return np.pad(im, ((0, pad_n), (0, pad_m)), mode="edge")

def run_optical_flow(frame0_path, frame1_path,
                     lam=1.0, sigma=1.0,
                     tol=1e-8, maxit=2000):
    """Original CG-based optical flow driver (no multigrid)."""
    I0 = mpimg.imread(frame0_path).astype(float)
    I1 = mpimg.imread(frame1_path).astype(float)
    if I0.ndim == 3:
        I0 = np.mean(I0, axis=2)
        I1 = np.mean(I1, axis=2)

    I0 = gaussian_filter(I0, sigma)
    I1 = gaussian_filter(I1, sigma)

    Ix, Iy = derivatives(I0, I1)
    It = I1 - I0

    rhsu, rhsv = build_right_hand_side(Ix, Iy, It)
    u0, v0 = np.zeros_like(I0), np.zeros_like(I0)

    u, v, info = OF_cg(u0, v0, Ix, Iy, lam, rhsu, rhsv, tol, maxit, verbose=True)

    print(f"CG finished in {info['it']} iterations, "
          f"relres={info['relres']:.2e}, converged={info['converged']}")
    return u, v, info

def run_optical_flow_pcg(frame0_path, frame1_path,
                         lam=1.0, sigma=1.0,
                         tol=1e-8, maxit=2000,
                         s1=2, s2=2, max_level=3):
    """
    Optical flow driver using PCG with multigrid V-cycle preconditioner
    and 2×2 (1/4) restriction stencil.
    """
    I0 = mpimg.imread(frame0_path).astype(float)
    I1 = mpimg.imread(frame1_path).astype(float)
    if I0.ndim == 3:
        I0 = np.mean(I0, axis=2)
        I1 = np.mean(I1, axis=2)

    I0 = gaussian_filter(I0, sigma)
    I1 = gaussian_filter(I1, sigma)

    # Pad so each dimension is divisible by 2**max_level
    I0 = pad_to_multiple_of_2L(I0, max_level)
    I1 = pad_to_multiple_of_2L(I1, max_level)

    Ix, Iy = derivatives(I0, I1)
    It = I1 - I0

    rhsu, rhsv = build_right_hand_side(Ix, Iy, It)
    u0, v0 = np.zeros_like(I0), np.zeros_like(I0)

    u, v, info = OF_pcg(
        u0, v0, Ix, Iy, lam, rhsu, rhsv,
        tol=tol, maxit=maxit,
        s1=s1, s2=s2, max_level=max_level,
        verbose=True
    )

    print(f"PCG+MG finished in {info['it']} iterations, "
          f"relres={info['relres']:.2e}, converged={info['converged']}")
    return u, v, info


if __name__ == "__main__":
    frame0_path = "/Users/ethanclement/Documents/NLAProject/frame10.png"
    frame1_path = "/Users/ethanclement/Documents/NLAProject/frame11.png"

    u, v, info = run_optical_flow(frame0_path, frame1_path)
    u_pcg, v_pcg, info_pcg = run_optical_flow_pcg(
        frame0_path,
        frame1_path,
        lam=1.0,
        sigma=1.0,
        tol=1e-8,
        maxit=2000,
        s1=2,
        s2=2,
        max_level=3,
    )


it=  10 relres=1.436e+00
it=  20 relres=1.331e+00
it=  30 relres=1.210e+00
it=  40 relres=1.052e+00
it=  50 relres=8.778e-01
it=  60 relres=7.130e-01
it=  70 relres=5.967e-01
it=  80 relres=4.992e-01
it=  90 relres=4.154e-01
it= 100 relres=3.554e-01
it= 110 relres=3.082e-01
it= 120 relres=2.619e-01
it= 130 relres=2.096e-01
it= 140 relres=1.803e-01
it= 150 relres=1.507e-01
it= 160 relres=1.246e-01
it= 170 relres=9.923e-02
it= 180 relres=8.198e-02
it= 190 relres=7.041e-02
it= 200 relres=5.994e-02
it= 210 relres=5.189e-02
it= 220 relres=4.611e-02
it= 230 relres=3.973e-02
it= 240 relres=3.315e-02
it= 250 relres=2.804e-02
it= 260 relres=2.301e-02
it= 270 relres=1.887e-02
it= 280 relres=1.565e-02
it= 290 relres=1.289e-02
it= 300 relres=1.056e-02
it= 310 relres=8.685e-03
it= 320 relres=7.409e-03
it= 330 relres=6.280e-03
it= 340 relres=5.562e-03
it= 350 relres=4.698e-03
it= 360 relres=3.862e-03
it= 370 relres=3.240e-03
it= 380 relres=2.856e-03
it= 390 relres=2.458e-03
it= 400 relres=2.074e-03
